# 카이사르 암호화 알고리즘
- 카이사르 인코더/디코더 구현(둘은 동일)
- 암호 해독 구현  
    동작 방식
    1. PorterStemmer 모듈 사용하여 단어 원형 추출
    2. 저장된 단어 목록에 단어 원형이 존재하는지 비교
    3. 3개 이상의 단어가 존재하면 맞는 문장으로 판단
- 랜덤 문장 생성 기능 구현
- 과제 수행
    1. words_alpha(mit).txt 파일에서 단어 7 ~ 10개 뽑아 문장(?) 생성
    2. 문장을 1 ~ 25로 caesar shift
    3. 해당 문장을 해독

사용하는 단어 출처( words_alpha(mit).txt 파일의 단어들 ): https://www.mit.edu/~ecprice/wordlist.10000  
  
사용된 환경
- vscode ipynb 환경
- python 버전 3.11.2 (2023.03.29 기준 파이썬 최신 버전)
- jupyter notebook 없이 파이썬 환경을 vscode와 바로 연결
- nltk 모듈 설치(pip install nltk)

## 1. 카이사르 암호화 인코더 구현
기본적으로 카이사르 암호는 인코딩 방식과 디코딩 방식이 완전히 동일하므로 대문자 및 소문자를 돌리는 방식만 구현하면 된다.  
이때, 암호는 다음 조건을 만족한다.
1. 대문자와 소문자만 순서 교체 대상으로 한정한다.
2. 대문자는 대문자끼리, 소문자는 소문자끼리만 순서를 바꾼다.

In [13]:
# 대문자 = 65 ~ 90로 26개
# 소문자 = 97 ~ 122로 26개

def getRoundCh(cur: int, step: int, base: int, round: int = 26):
    """
    @param cur 문자의 인덱스 값
    @param step 카이사르 움직일 거리
    @param base 기본 값(ex 대문자는 65)
    @return step만큼 이동한 문자
    """
    assert(0 <= (cur - base) <= round) # base-cur 값은 라운드 내에 있어야 함
    return chr(base + (cur - base + step) % round)

def encoder(string: str, step: int = 1, direction: bool = True):
    """
    @param string 암호화 대상이 되는 문자열
    @param step 이동할 수치
    @param direction 이동하는 방향. True이면 인덱스가 증가하는 방향
    @return 카이사르 암호화가 적용된 문자열
    """
    if not direction: #반대방향인 경우
        step *= -1 
    result: str = ""
    for ch in string:
        code = ord(ch)
        if 65 <= code <= 90: # 대문자라면
            result += getRoundCh(code, step, 65)
        elif 97 <= code <= 122:
            result += getRoundCh(code, step, 97)
        else:
            result += ch
    
    return result

In [14]:
# 테스팅 코드. 정상적으로 동작함!
print(encoder("Hello, World!", 2))
print(encoder("Hello, World!", 2, False))

Jgnnq, Yqtnf!
Fcjjm, Umpjb!


## 2. 암호 해독 구현
현재 과제에서는 컴퓨터 자신이 암호문을 스스로 복호화할 수 있어야 하므로, 현재 문장의 단어들이 실제로 존재하는지 판단하기 위한 방법이 필요하다.  
mit에서 제공하는 10000개의 단어에 대한 어간을 저장해놓은 후, 현재 들어오는 단어의 어간을 분석, 해당 단어 풀에 존재하는지 체크하여 3개 단어 이상 맞으면 해결했다고 간주한다.

### nltk 토크나이저 동작과 re 모듈의 substitution 동작 방식 확인

In [15]:
import nltk
import re

my_string = "hello, my name is heejun!"
cut_str = re.sub(r"[^0-9A-Za-z]"," ", my_string) # 특문 제거

# 정규식으로 특수 문자 제거
# 특수 문자는 어차피 카이사르 암호에서 중요한 정보가 아니므로, 제거 가능

result = nltk.tokenize.word_tokenize(cut_str)
print(result)

['hello', 'my', 'name', 'is', 'heejun']


### PorterStemmer 동작 확인

In [16]:
from nltk.stem import PorterStemmer # 어간을 추출하기 위한 라이브러리
from nltk.tokenize import word_tokenize
import re

contents: list[str] = []
with open('test1.txt','r',encoding='utf-8') as file:
    for line in file:
        contents.append(line)

stemmer = PorterStemmer()
for content in contents:
    stemwords:list[str] = []
    content = re.sub(r"[^0-9A-Za-z]"," ", content)
    for word in word_tokenize(content):
        stemmed = stemmer.stem(word = word)
        stemwords.append(stemmed)
    print(stemwords) # 정상적으로 어간을 추출하고 있음.

['i', 'know', 'a', 'young', 'man', 'who', 'declar', 'that', 'after', 'read', 'a', 'certain']
['explor', 's', 'descript', 'of', 'a', 'journey', 'across', 'the', 'burn', 'sahara', 'he']
['found', 'to', 'hi', 'amaz', 'that', 'hi', 'nose', 'wa', 'cover', 'with', 'freckl', 'the']
['reader', 'will', 'perhap', 'rememb', 'how', 'on', 'some', 'raini', 'day', 'in', 'hi', 'childhood']
['he', 'ha', 'sat', 'over', 'the', 'fire', 'and', 'ha', 'read', 'sea', 'stori', 'and', 'dream']
['sea', 'dream', 'until', 'hi', 'lip', 'he', 'will', 'swear', 'have', 'tast', 'salt', 'ala']
['one', 's', 'littl', 'agil', 'in', 'the', 'art', 'of', 'narrat', 'is', 'wholli', 'inadequ']
['for', 'the', 'product', 'at', 'thi', 'time', 'of', 'life', 'of', 'ani', 'such', 'phenomena']
['upon', 'the', 'gentl', 'skin', 'of', 'those', 'who', 'chanc', 'to', 'read', 'these', 'page', 'were']
['one', 'a', 'master', 'maker', 'of', 'literatur', 'one', 'might', 'herewith', 'lead', 'the']
['imagin', 'so', 'straight', 'into', 'the', 'bois

### mit에서 제공하는 단어 목록의 어간을 추출하여 파일에 저장

In [20]:
from nltk.stem import PorterStemmer
stems: set[str] = set()
stemmer = PorterStemmer()
with open('words_alpha(mit).txt','r',encoding='utf-8') as rfile:
    count = 0
    for line in rfile: # 파일에서 읽기
        result = stemmer.stem(line.strip()) # 띄어쓰기 지우기
        stems.add(result)
print(f"stems length: {len(stems)}")

with open('words_stems(mit).txt','a', encoding='utf-8') as wfile:
    wfile.write(",".join(stems))
# 1000개의 단어에서 6783개의 어간 추출

stems length: 6783


### 추출한 어간 목록을 이용하여 크래킹 시도하는 모듈 작성

In [25]:
class CaesarCipher:
        # 대문자 = 65 ~ 90로 26개
    # 소문자 = 97 ~ 122로 26개
    def getRoundCh(cur: int, step: int, base: int, round: int = 26):
        """
        @param cur 문자의 인덱스 값
        @param step 카이사르 움직일 거리
        @param base 기본 값(ex 대문자는 65)
        @return step만큼 이동한 문자
        """
        assert(0 <= (cur - base) <= round) # base-cur 값은 라운드 내에 있어야 함
        return chr(base + (cur - base + step) % round)

    def encoder(string: str, step: int = 1, direction: bool = True):
        """
        @param string 암호화 대상이 되는 문자열
        @param step 이동할 수치
        @param direction 이동하는 방향. True이면 인덱스가 증가하는 방향
        @return 카이사르 암호화가 적용된 문자열
        """
        if not direction: #반대방향인 경우
            step *= -1 
        result: str = ""
        for ch in string:
            code = ord(ch)
            if 65 <= code <= 90: # 대문자라면
                result += getRoundCh(code, step, 65)
            elif 97 <= code <= 122:
                result += getRoundCh(code, step, 97)
            else:
                result += ch
        
        return result

In [41]:
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re

class CaesarCracker:
    __stems: set[str] ## 미리 준비된 어간 목록
    __stemmer : PorterStemmer

    def __init__(self, stem_source: str):
        self.__stemmer = PorterStemmer()
        self.__stems = set() # 초기화
        with open(stem_source,'r',encoding='utf-8') as file:
            for line in file:
                stems = line.split(',') # ,로 stem 분할
                for stem in stems:
                    self.__stems.add(stem) # 분할된 stem 목록 삽입
    
    def genToken(string: str, step: int):
        """
        입력 받은 문자열을 토큰으로 나눠 반환한다.
        @param string 입력 받은 문자열
        @param step 문자를 이동하는 수치
        """
        caesared_str = CaesarCipher.encoder(string,step)
        processed_str = re.sub(r"[^0-9A-Za-z]"," ", caesared_str) # 특문 제거
        tokens = word_tokenize(processed_str) # 생성된 토큰들
        return (caesared_str, tokens)
                
    def crack(self, string: str, max_round: int = 26, criteria: int = 3):
        """
        해독한 문자열에 대해 criteria을 초과하는 문장 목록을 반환한다. criteria 이상은 검사하지 않는다.
        @param string 크래킹 대상이 되는 문자열
        @param max_round 최대 라운드 수
        @param criteria 현재 문장이 맞다고 판단할 어간 일치 수
        @return list[(round: int, result: str)] 해독한 문장들에 대해 criteria을 만족하는 문자열들의 라운드 및 결과 문장
        """
        result = []
        for i in range(1,max_round):
            caesared_str, tokens = CaesarCracker.genToken(string, i)
            count = 0
            for token in tokens:
                stem = self.__stemmer.stem(token)
                if stem in self.__stems:
                    count += 1 # 현재 문장을 맞다고 판단.
                    if count >= criteria:
                        result.append((i, caesared_str)) # 문장을 삽입
                        break
            
        return result if result else [(-1, "Not Found")] # 발견하지 못했다고 표현

    def crackFullSearch(self, string: str, max_round: int = 26, criteria: int = 3):
        """
        문장 내 단어를 모두 stem과 비교하여 criteria을 넘는 문자열 목록을 최대 유사도 순서로 반환한다.
        @param string 크래킹 대상이 되는 문자열
        @param max_round 최대 라운드 수
        @param criteria 현재 문장이 맞다고 판단할 어간 일치 수
        @return list[tuple[round: int, result: str,count: int]] criteria 이상의 count을 가지는 문자열을 count가 큰 순서로 정렬해서 반환
        """
        result: list[tuple[int,str,int]] = []
        for i in range(1,max_round):
            caesared_str, tokens = CaesarCracker.genToken(string, i)
            count = 0
            for token in tokens:
                stem = self.__stemmer.stem(token)
                if stem in self.__stems:
                    count += 1 # 현재 문장을 맞다고 판단.
            if count >= criteria:
                result.append((i, caesared_str, count)) # 문장을 삽입
        result.sort(key = lambda it: it[2], reverse=True)
        return result if result else [(-1, "Not Found")] # 발견하지 못했다고 표현
    


### 크래킹 모듈 동작 테스트

In [42]:
cracker = CaesarCracker("words_stems(mit).txt")

encrypted1 = CaesarCipher.encoder("hello, world! this world is mine!", 3)
print(cracker.crack(encrypted1))
# 오, 잘 된다!

encrypted2 = CaesarCipher.encoder("I know a young man who declares that after reading a certain", 7)
print(cracker.crack(encrypted2,criteria=3)) # criteria = 3인 경우, 제대로 된 답이 나오지 않는다.

[(23, 'hello, world! this world is mine!')]
[(1, 'Q svwe i gwcvo uiv epw lmktizma bpib inbmz zmilqvo i kmzbiqv'), (2, 'R twxf j hxdwp vjw fqx mnlujanb cqjc jocna anjmrwp j lnacjrw'), (3, 'S uxyg k iyexq wkx gry nomvkboc drkd kpdob boknsxq k mobdksx'), (4, 'T vyzh l jzfyr xly hsz opnwlcpd esle lqepc cplotyr l npcelty'), (5, 'U wzai m kagzs ymz ita pqoxmdqe ftmf mrfqd dqmpuzs m oqdfmuz'), (6, 'V xabj n lbhat zna jub qrpynerf gung nsgre ernqvat n pregnva'), (7, 'W ybck o mcibu aob kvc rsqzofsg hvoh othsf fsorwbu o qsfhowb'), (8, 'X zcdl p ndjcv bpc lwd strapgth iwpi puitg gtpsxcv p rtgipxc'), (9, 'Y adem q oekdw cqd mxe tusbqhui jxqj qvjuh huqtydw q suhjqyd'), (10, 'Z befn r pflex dre nyf uvtcrivj kyrk rwkvi ivruzex r tvikrze'), (11, 'A cfgo s qgmfy esf ozg vwudsjwk lzsl sxlwj jwsvafy s uwjlsaf'), (12, 'B dghp t rhngz ftg pah wxvetkxl matm tymxk kxtwbgz t vxkmtbg'), (13, 'C ehiq u sioha guh qbi xywfulym nbun uznyl lyuxcha u wylnuch'), (14, 'D fijr v tjpib hvi rcj yzxgvmzn ocvo vaozm mzvyd

In [44]:
#발견하면 바로 나오기
print(cracker.crack(encrypted2,criteria=3))
print(cracker.crack(encrypted2,criteria=4))
print(cracker.crack(encrypted2,criteria=5))

#full search -> 빈도가 높은 순으로 뽑아주기
print(cracker.crackFullSearch(encrypted2, criteria=3))
print(cracker.crackFullSearch(encrypted2, criteria=4))
print(cracker.crackFullSearch(encrypted2, criteria=5))

#일반적인 문장에도 성립한다.


[(1, 'Q svwe i gwcvo uiv epw lmktizma bpib inbmz zmilqvo i kmzbiqv'), (2, 'R twxf j hxdwp vjw fqx mnlujanb cqjc jocna anjmrwp j lnacjrw'), (3, 'S uxyg k iyexq wkx gry nomvkboc drkd kpdob boknsxq k mobdksx'), (4, 'T vyzh l jzfyr xly hsz opnwlcpd esle lqepc cplotyr l npcelty'), (5, 'U wzai m kagzs ymz ita pqoxmdqe ftmf mrfqd dqmpuzs m oqdfmuz'), (6, 'V xabj n lbhat zna jub qrpynerf gung nsgre ernqvat n pregnva'), (7, 'W ybck o mcibu aob kvc rsqzofsg hvoh othsf fsorwbu o qsfhowb'), (8, 'X zcdl p ndjcv bpc lwd strapgth iwpi puitg gtpsxcv p rtgipxc'), (9, 'Y adem q oekdw cqd mxe tusbqhui jxqj qvjuh huqtydw q suhjqyd'), (10, 'Z befn r pflex dre nyf uvtcrivj kyrk rwkvi ivruzex r tvikrze'), (11, 'A cfgo s qgmfy esf ozg vwudsjwk lzsl sxlwj jwsvafy s uwjlsaf'), (12, 'B dghp t rhngz ftg pah wxvetkxl matm tymxk kxtwbgz t vxkmtbg'), (13, 'C ehiq u sioha guh qbi xywfulym nbun uznyl lyuxcha u wylnuch'), (14, 'D fijr v tjpib hvi rcj yzxgvmzn ocvo vaozm mzvydib v xzmovdi'), (15, 'E gjks w ukqjc iwj sdk

## 3. 랜덤 문장 생성 클래스 구현
words_alpha(mit).txt 파일에서 7 ~ 10개 단어를 뽑아 랜덤 문장을 생성.  
문장 자체는 말이 안될 수도 있지만, 단어는 모두 유효.

In [48]:
from random import randint

class RSGenerator:
    __words: list[str]
    def __init__(self, word_source: str):
        """
        @param word_source 단어가 담긴 파일 이름. 단어는 한 줄에 하나 있는 것을 기준으로 한다.
        """
        self.__words = []
        with open(word_source,'r',encoding='utf-8') as file:
            self.__words.extend([line.strip() for line in file])
    
    def gen(self, min_size:int = 7, max_size:int = 10):
        """
        start ~ stop 길이 사이의 단어를 포함한 문장을 반환한다.
        @param min_size 최소 길이
        @param max_size 최대 길이       
        """
        arr = []
        wlen = len(self.__words)
        for _ in range(randint(min_size,max_size)):
            word = self.__words[randint(0, wlen - 1)]
            # 맨 마지막 숫자 걸리면 인덱스 바깥
            arr.append(word)
        return " ".join(arr)

### 실행 테스트

In [49]:
generator = RSGenerator('words_alpha(mit).txt')
for i in range(10):
    print(generator.gen())

classes petersburg limousines sue gotten webcams brush equation stylus
weekends huge milfs giant assumed aluminium catch interested had
compared troops feat gore bigger taylor unless forecasts league
k reads fotos citizens designation sacramento reproduction plates restored wrapping
typically begin hawk clients regular plot angola
trailer pathology filled violations completed tones dynamics clicks
boston hearing label wall angola nonprofit tip computing
config simulations beginner scoop erotic quad oldest eds
continued dig soonest there base garlic strikes
salaries violation november conferencing understood adaptive ecuador cdt stick shit


## 4. 과제 수행
1. 단어 7 ~ 10개인 문장 생성
2. 생성된 단어를 1 ~ 25사이로 shift
3. 카이사르 암호를 해독(100회)

In [51]:
from random import randint
from time import time_ns
generator = RSGenerator('words_alpha(mit).txt')
cracker = CaesarCracker('words_stems(mit).txt')

times: list[int] = []

for idx in range(100): # 100회 수행
    sentence = generator.gen() # 문장 생성
    encrypted = CaesarCipher.encoder(sentence,randint(1,25)) # 문장 생성
    start = time_ns() # 시작 시간
    result = cracker.crack(encrypted)
    timelapse = time_ns() - start
    print(f"[{idx}] TimeLapse: {timelapse}, result: {result}")
    times.append(timelapse)

avg = sum(times) / len(times) # 평균 나노초

print(f"average time is: {avg}ns, is {avg/1000000}ms")

[0] TimeLapse: 8999800, result: [(13, 'watch posts idaho template incidents preserve ahead priced prizes')]
[1] TimeLapse: 9999000, result: [(12, 'computation updating sofa hungry consist connected brutal gloves lead options')]
[2] TimeLapse: 8015500, result: [(24, 'commitment frequencies laden recordings subsidiaries finest monster neural subscribe')]
[3] TimeLapse: 12987300, result: [(21, 'utah raising sucks domestic saudi fig fruits graphs ghz')]
[4] TimeLapse: 33000100, result: [(11, 'spread municipal certainly spies girls erp tapes lasting')]
[5] TimeLapse: 20003100, result: [(15, 'wanting improving austin wonderful candidates xxx northwest')]
[6] TimeLapse: 20997200, result: [(14, 'bookmarks cathedral buyers turner logged boss bouquet going nec election')]
[7] TimeLapse: 18999500, result: [(20, 'hire eyes francisco democratic can finder newport jo downloading')]
[8] TimeLapse: 12002400, result: [(12, 'des week xp plug lookup sand fc healthy eastern partnerships')]
[9] TimeLapse: 